# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [2]:
from scipy import stats as st 
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [3]:
#Сохраним данные о трех регионах в свои переменные
file_name_1 = '/datasets/geo_data_0.csv'
file_name_2 = '/datasets/geo_data_1.csv'
file_name_3 = '/datasets/geo_data_2.csv'
geo_data_0 = pd.read_csv(file_name_1)
geo_data_1 = pd.read_csv(file_name_2)
geo_data_2 = pd.read_csv(file_name_3)

*Проведем небольшую проверку на чистоту в данных*

In [4]:
#Проверим данные на пропуски
if (geo_data_0.isnull().sum().sum() + geo_data_1.isnull().sum().sum() + geo_data_2.isnull().sum().sum()) ==0:
    print('В данных пропусков нет')

В данных пропусков нет


In [5]:
#Проверим данные на наличие дубликатов
if (geo_data_2.duplicated().sum() + geo_data_1.duplicated().sum() + geo_data_2.duplicated().sum()) == 0:
    print('дубликатов не обнаружено')
else:
    print('обнаружены дубликаты')

дубликатов не обнаружено


## Обучение и проверка модели

In [6]:
#Напишем функцию деления данных на обучающую и валидационную выборки в соотношении 75:25

def data_split(geo_data):
    
    X = geo_data[['f0', 'f1', 'f2']]
    y = geo_data['product']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12345)
    print (X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
    
    return X_train, X_valid, y_train, y_valid

In [7]:
#Поделим данные для 3-х регионов и проверим правильность их разделения

i = 0

for geo_data in [geo_data_0, geo_data_1, geo_data_2]:
    
    globals()['X_train_' + str(i)], globals()['X_valid_' + str(i)], \
    globals()['y_train_' + str(i)], globals()['y_valid_' + str(i)] = data_split(geo_data)
    i += 1

(75000, 3) (75000,) (25000, 3) (25000,)
(75000, 3) (75000,) (25000, 3) (25000,)
(75000, 3) (75000,) (25000, 3) (25000,)


In [8]:
def model_learning (X_train, X_valid, y_train, y_valid):
 
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_predict = model.predict(X_valid)
    y_valid = y_valid.reset_index(drop=True)
    predictions = cross_val_predict(model, X_valid, y_valid, cv=5)
    
    # Кросс-валидация с разбитием на 5 блоков, получение метрики качества (R2) для каждой из них 
    scores = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1)
    print ('R2 региона на кросс-валидации:', scores)    
    
    # Среднее значение метрики и стандартное отклонение
    print (f'Средний R2 на кросс-валидации: {np.mean(scores):.5f} std: {np.std(scores):.5f}')
        
    # Аccuracy-метрика для предсказаний на лучшем блокее кросс-валидации
    print (f'Лучший R2: {r2_score(y_valid, predictions):.4f}')
    
    # RMSE для предсказаний на лучшем блоке кросс-валидации
    print (f'RMSE: {mean_squared_error(y_valid, predictions, squared=False):.2f}')
        
    print (F'Предсказанный средний запас: {y_predict.mean():.2f}')
    print (F'Истинный средний запас: {y_valid.mean():.2f}')
    
    return y_predict, y_valid

In [9]:
print ('Качество модели для 1 региона: \n')
y_predict_0, y_valid_0 = model_learning(X_train_0, X_valid_0, y_train_0, y_valid_0)

Качество модели для 1 региона: 

R2 региона на кросс-валидации: [0.26543147 0.27779784 0.28135147 0.27476308 0.27108881]
Средний R2 на кросс-валидации: 0.27409 std: 0.00549
Лучший R2: 0.2801
RMSE: 37.57
Предсказанный средний запас: 92.59
Истинный средний запас: 92.08


In [10]:
print ('Качество модели для 2 региона: \n')
y_predict_1, y_valid_1 = model_learning(X_train_1, X_valid_1, y_train_1, y_valid_1)

Качество модели для 2 региона: 

R2 региона на кросс-валидации: [0.99963474 0.99962007 0.99962403 0.99962336 0.99962121]
Средний R2 на кросс-валидации: 0.99962 std: 0.00001
Лучший R2: 0.9996
RMSE: 0.89
Предсказанный средний запас: 68.73
Истинный средний запас: 68.72


In [11]:
print ('Качество модели для 3 региона: \n')
y_predict_2, y_valid_2 = model_learning(X_train_2, X_valid_2, y_train_2, y_valid_2)

Качество модели для 3 региона: 

R2 региона на кросс-валидации: [0.19754075 0.19883786 0.19447795 0.19719633 0.19435838]
Средний R2 на кросс-валидации: 0.19648 std: 0.00177
Лучший R2: 0.2052
RMSE: 40.03
Предсказанный средний запас: 94.97
Истинный средний запас: 94.88


***Вывод***

Разбили данные на обучающую и валидационную выборки в соотношении 75к25. Обучили модели, а так же сделали предсказания на валидационной выборке. Вывели средний запас предсказанного сырья и RMSE моделей.

Обученные модели показали хорошие результаты, особенно для 2-го региона.

## Подготовка к расчёту прибыли

Из условий задачи следует, что:

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
Доход с каждой единицы продукта составляет 450 тыс. рублей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5% т.е. вероятность получить 10 млрд рублей дохода с 200 лучших больше 95%, так как распределение двухстороннее. Среди трех выбрать регион с наибольшей средней прибылью.

In [13]:
#Для всех ключевых значений создадим константы
BUDGET = 10000000000
BEST_LOC_NUM = 200
LOC_NUM = 500
PRODUCT_UNIT_INCOME = 450000
ALPHA = 0.025
BOOTSTRAP_SAMPLES = 1000

In [14]:
#Посчитаем минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
min_mean_product = BUDGET / BEST_LOC_NUM / PRODUCT_UNIT_INCOME
min_mean_product

111.11111111111111

Нам нужно выбрать регионы, в которых среднее количество продукта у 200 лучших скважин по объему сырья  было не меньше найденного с заявленной вероятностью чтобы гарантированно окупить бюджет

In [34]:
#Напишем функцию расчёта прибыли
def profit(y_predict_sample, y_valid):
    
    y_predict_sample_best_200 = y_predict_sample.sort_values(ascending=False)[:BEST_LOC_NUM]
    sample_profit = y_valid[y_predict_sample_best_200.index].sum() * PRODUCT_UNIT_INCOME - BUDGET
    
    return sample_profit

## Расчёт прибыли и рисков 

In [35]:
#Напишем функцию для просчета суммарной прибыли у 200 из 500 лучших скважин в 1000 случайных выборок 
def sample_profit_dist(y_predict, y_valid):
    
    state = RandomState(12345)
    sample_profit_distrib = []
    count = 0      #где count это счетчик числа убыточных выборок
    
    for i in range(BOOTSTRAP_SAMPLES):    
        
        #Применим Bootstrap
        y_predict_sample = pd.Series(y_predict).sample(n=LOC_NUM, replace=True, random_state=state)
        
        
        #Посчитаем прибыль для выборки
        sample_profit = profit(y_predict_sample, y_valid)
        
        #Создадим распределение значений прибыли выборок
        sample_profit_distrib.append(sample_profit)
        
        # Посчитаем убыточные выборки
        if sample_profit < 0:
            count += 1
    
    return pd.Series(sample_profit_distrib), count

In [36]:
#Получим распределениу выборок прибыли по каждому региону
sample_profit_dist_0, count_0 = sample_profit_dist(y_predict_0, y_valid_0)
sample_profit_dist_1, count_1 = sample_profit_dist(y_predict_1, y_valid_1)
sample_profit_dist_2, count_2 = sample_profit_dist(y_predict_2, y_valid_2)

In [37]:
#Напишем функцию для получения показателей средней прибыли, риска убытков и выводов по регионам

def result(profit, count):
    
    print (f'Средняя прибыль: {profit.mean() / 1000000:.2f} млн руб.')    
    print (f'95%-й доверительный интервал: {np.quantile(profit, 0.025):.0f}, {np.quantile(profit, 0.975):.0f}')
    
    # Расчет доли убыточных выборок
    p_value = 1. * count / BOOTSTRAP_SAMPLES
    
    if p_value < ALPHA:
        print(f'Вероятность убытков: {p_value:.2%} - меньше допустимой, следовательно регион подойдет для разработки')
    else:
        print(f'Вероятность убытков: {p_value:.2%} - больше допустимой, следовательно регион не подойдет для разработки')

In [38]:
print ('Показатели региона 1: \n')
result(sample_profit_dist_0, count_0)

Показатели региона 1: 

Средняя прибыль: 396.16 млн руб.
95%-й доверительный интервал: -111215546, 909766942
Вероятность убытков: 6.90% - больше допустимой, следовательно регион не подойдет для разработки


In [39]:
print ('Показатели региона 2: \n')
result(sample_profit_dist_1, count_1)

Показатели региона 2: 

Средняя прибыль: 456.05 млн руб.
95%-й доверительный интервал: 33820509, 852289454
Вероятность убытков: 1.50% - меньше допустимой, следовательно регион подойдет для разработки


In [40]:
print ('Показатели региона 3: \n')
result(sample_profit_dist_2, count_2)

Показатели региона 3: 

Средняя прибыль: 404.40 млн руб.
95%-й доверительный интервал: -163350413, 950359575
Вероятность убытков: 7.60% - больше допустимой, следовательно регион не подойдет для разработки


***Вывод***

Таким образом можно сделать вывод что оценка рисков позволяет нам оставить только один регион под номером 2 в котором вероятность убытков 1.50% что меньше допустимого 2.5%.